In [1]:
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src')
from features.get_bert_embeddings import BertFeatureExtractor


2024-11-19 11:19:22.324122: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 11:19:22.333754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731986362.346719   14999 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731986362.350061   14999 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-19 11:19:22.363043: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [5]:
!ls ../data/processed/sat_train/


train_chunked_processed.csv  train_full_processed.csv


In [66]:
chunked_fp = '../data/processed/sat_train/train_chunked_processed.csv'
df = pd.read_csv(chunked_fp)

In [67]:
df.head()

,essay_id,essay_set,essay,rescaled_score,low_med_hi,low_med_hi_numeric
0,10135,4,The reason why the author concludes the story ...,7.0,medium,2
1,4131,2,Would you censor the @CAPS1? i @CAPS2 i sure w...,4.6,low,1
2,4131,2,Throught out the story the @CAPS5 gives the bo...,4.6,low,1
3,4131,2,Why?! The @CAPS1 for example is just a book fu...,4.6,low,1
4,4131,2,Thanks to a close friend. @CAPS8 could also be...,4.6,low,1


In [68]:
# Setting up a results df with all predictions

In [69]:
df.columns

Index(['essay_id', 'essay_set', 'essay', 'rescaled_score', 'low_med_hi',
       'low_med_hi_numeric'],
      dtype='object')

In [70]:
df['rounded_rescaled_score'] = df['rescaled_score'].round()

In [71]:
df_results = df[['essay_id', 'essay_set', 'rescaled_score', 'low_med_hi','low_med_hi_numeric', 'rescaled_score']]

## Setting up cross validation

In [81]:
n = round(df.shape[0]/5)
n

3507

In [95]:
unique_essay_ids = df['essay_id'].value_counts().index.values
np.random.shuffle(unique_essay_ids)
unique_essay_ids

array([ 3073,  4254,  3343, ...,  3159,  7266, 12438])

In [104]:
chunks = np.array_split(unique_essay_ids, 5)
chunks

[array([  154,   585, 19317, ...,  3036,  6908,    42]),
 array([13344,  3659, 16542, ..., 19289, 15871,   484]),
 array([18672, 15040,  1528, ...,  9074,   333,  9833]),
 array([15955,  3464,   677, ..., 18489, 12293,  3654]),
 array([ 6200, 19028, 21415, ...,  1233, 16222, 15461])]

In [109]:
df[df['essay_id'].isin(chunks[0])].index.values

array([    0,    11,    20, ..., 17515, 17516, 17536])

In [110]:
folds = {}

In [114]:
df['chunk'] = 99

In [120]:
for i,chunk in enumerate(chunks):
    idx = df[df['essay_id'].isin(chunk)].index.values
    df.loc[idx,'chunk']=i
    # folds[i] = idx

In [124]:
df.drop(columns='chunl', inplace=True)

In [127]:
df.groupby('chunk').count()

,essay_id,essay_set,essay,rescaled_score,low_med_hi,low_med_hi_numeric,rounded_rescaled_score
chunk,,,,,,,
0,3465,3465,3465,3465,3465,3465,3465
1,3481,3481,3481,3481,3481,3481,3481
2,3509,3509,3509,3509,3509,3509,3509
3,3605,3605,3605,3605,3605,3605,3605
4,3477,3477,3477,3477,3477,3477,3477


In [128]:
chunked_fp = '../data/processed/sat_train/train_chunked_processed.csv'

In [129]:
df.to_csv(chunked_fp, index=False)

## Training 

In [131]:
pd.read_csv(chunked_fp)

,essay_id,essay_set,essay,rescaled_score,low_med_hi,low_med_hi_numeric,rounded_rescaled_score,chunk
0,10135,4,The reason why the author concludes the story ...,7.0,medium,2,7.0,0
1,4131,2,Would you censor the @CAPS1? i @CAPS2 i sure w...,4.6,low,1,5.0,2
2,4131,2,Throught out the story the @CAPS5 gives the bo...,4.6,low,1,5.0,2
3,4131,2,Why?! The @CAPS1 for example is just a book fu...,4.6,low,1,5.0,2
4,4131,2,Thanks to a close friend. @CAPS8 could also be...,4.6,low,1,5.0,2
...,...,...,...,...,...,...,...,...
17532,13151,5,The mood created by the author in this memoir ...,5.5,medium,2,6.0,3
17533,3061,2,"Dear Katherine Paterson, Do you believe that c...",4.6,low,1,5.0,2
17534,3061,2,Either way I think that they shouldn't remove ...,4.6,low,1,5.0,2
17535,3061,2,Just because that single person doesn't like t...,4.6,low,1,5.0,2
